In [ ]:
%load_ext autoreload
import numpy as np
import seaborn as sns
import os
import GallenModel as ClassificationModelsimple
import geopandas as gpd
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.utils import resample

In [ ]:

def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  df = dataframe.copy()
  labels = df.pop('Landslide')
  df = {key: value.to_numpy()[:,tf.newaxis] for key, value in dataframe.items()}
  ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  ds = ds.prefetch(batch_size)
  return ds

def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
  # Create a layer that turns strings into integer indices.
  if dtype == 'string':
    index = layers.StringLookup(max_tokens=max_tokens)
  # Otherwise, create a layer that turns integer values into integer indices.
  else:
    index = layers.IntegerLookup(max_tokens=max_tokens)

  # Prepare a `tf.data.Dataset` that only yields the feature.
  feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the set of possible values and assign them a fixed integer index.
  index.adapt(feature_ds)

  # Encode the integer indices.
  encoder = layers.CategoryEncoding(num_tokens=index.vocabulary_size())

  # Apply multi-hot encoding to the indices. The lambda function captures the
  # layer, so you can use them, or include them in the Keras Functional model later.
  return lambda feature: encoder(index(feature))

def get_normalization_layer(name, dataset):
  # Create a Normalization layer for the feature.
  normalizer = layers.Normalization(axis=None)

  # Prepare a Dataset that only yields the feature.
  feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the statistics of the data.
  normalizer.adapt(feature_ds)

  return normalizer

In [ ]:
def trainmodel(model,train_ds,val_ds,NUMBER_EPOCHS = 100):
    
    
    filepath='TrainedWeightsCrossVal'
    BATCH_SIZE=32
    
    model_checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
        filepath,
        monitor="val_auc",
        verbose=0,
        save_best_only=True,
        save_weights_only=False,
        mode="max",
        save_freq="epoch",
        options=None
    )
    print(type(train_ds))
    hist = model.fit(train_ds,
                     epochs=NUMBER_EPOCHS,
                     batch_size=BATCH_SIZE,
                     validation_data=val_ds,
                    #  validation_split=0.2,#auto validate using 20% of random samples at each epoch
                     verbose=1, callbacks=[model_checkpoint_callback],class_weight = {0: 1, 1: 5}

                    )
    del model
    return hist


In [ ]:
def BootStrapGeotech(dfc):
  n_bootstrap=50
  i=0
  for i in range(n_bootstrap):
    print(i)
    all_inputs = []
    encoded_features = []
    
    #    df.iloc[train_index]
    # df.iloc[test_index]
    train_df= resample(dfc, random_state=None,n_samples=10000, replace=False)
    test_df=dfc[~dfc.cat.isin(train_df.cat)]
    print(f"Number of train set{len(train_df)} and number of test set {len(test_df)}")

    exai_ds=df_to_dataset(train_df[['Est_m','Nrt_m','HC_m','VC_m','Slp_m','Prc_m','NDVI_m','PGA_Usgs','Sand_m','Silt_m','Clay_m','Bdod_m','GLG','Landslide']])
    val_ds=df_to_dataset(test_df[['Est_m','Nrt_m','HC_m','VC_m','Slp_m','Prc_m','NDVI_m','PGA_Usgs','Sand_m','Silt_m','Clay_m','Bdod_m','GLG','Landslide']],shuffle=False)
    y_test=test_df['Landslide'].to_numpy()
    

    for header in numerical_cols:
      numeric_col = tf.keras.Input(shape=(1,), name=header)
      normalization_layer = get_normalization_layer(header, exai_ds)
      encoded_numeric_col = normalization_layer(numeric_col)
      all_inputs.append(numeric_col)
      encoded_features.append(encoded_numeric_col)

    for header in categorical_cols:
      categorical_col = tf.keras.Input(shape=(1,), name=header, dtype='string')
      encoding_layer = get_category_encoding_layer(name=header,
                                                  dataset=exai_ds,
                                                  dtype='string',
                                                  max_tokens=9)
      encoded_categorical_col = encoding_layer(categorical_col)
      all_inputs.append(categorical_col)
      encoded_features.append(encoded_categorical_col)

    clfmdl=ClassificationModelsimple.LandslideModel()
    clfmdl.getclassificationModel(all_inputs=all_inputs, encoded_features=encoded_features)
    clfmdl.getOptimizer()
    clfmdl.compileModel()

    trainmodel(clfmdl.model,exai_ds,val_ds,NUMBER_EPOCHS = 50)
    del clfmdl.model,clfmdl

    model =  tf.keras.models.load_model("TrainedWeightsCrossVal/")
    
    all_data=df_to_dataset(df[['Est_m','Nrt_m','HC_m','VC_m','Slp_m','Prc_m','NDVI_m','PGA_Usgs','Sand_m','Silt_m','Clay_m','Bdod_m','GLG','Landslide']],shuffle=False)
    geo_Tech1 = tf.keras.Model(inputs=model.input, outputs=model.get_layer("cohesion").output)*5 # apply linear scaling to compensate for 5x scaling on loss function
    geotech_preds=geo_Tech1.predict(all_data)
    np.save(f'GeotechResultsGallen/CoGeotech_{str(i)}.npy',geotech_preds)

    geo_Tech2 = tf.keras.Model(inputs=model.input, outputs=model.get_layer("internalFriction").output)
    geotech_preds=geo_Tech2.predict(all_data)
    np.save(f'GeotechResultsGallen/IfGeotech_{str(i)}.npy',geotech_preds)
    
    del geo_Tech2
    del geo_Tech1
    del model
    tf.keras.backend.clear_session()
    i+=1
    # del clfmdl

In [ ]:
categorical_cols = ['GLG']
numerical_cols=['Est_m', 'Nrt_m', 'HC_m', 'VC_m', 'Slp_m', 'Prc_m', 'NDVI_m', 'PGA_Usgs', 'Sand_m', 'Silt_m', 'Clay_m', 'Bdod_m']
df=gpd.read_file('Data/NepalEqUSGSGallen.gpkg')
BootStrapGeotech(df)

# Geotechnical uncertainity

In [ ]:
import sklearn.metrics
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.pyplot import figure
import numpy as np 
import geopandas as gpd
rcl_indexes=[]
df=gpd.read_file('Data/NepalEqUSGSGallen.gpkg')

In [ ]:
rv=np.load(f'GeotechResultsGallen/CoGeotech_{str(0)}.npy')

In [ ]:
co_all = np.empty((50,rv.shape[0]))
for i in range(50):
    rv=np.load(f'GeotechResultsGallen/CoGeotech_{str(i)}.npy')
    co_all[i]=rv

In [ ]:
if_all = np.empty((50,rv.shape[0]))
for i in range(1,11):
    rv=np.load(f'GeotechResultsGallen/IfGeotech_{str(i)}.npy')
    if_all[i]=rv

In [ ]:
import contextily as cx
from matplotlib_scalebar.scalebar import ScaleBar

fig,axs = plt.subplots(2,2,dpi=500, figsize=(10,10))
df_wm = df.to_crs(epsg=3857)
co_all[co_all==0.0] = np.nan
co_all_bak = co_all
df_wm['cohesion'] =np.nanmedian(co_all_bak, axis=0)
df_wm['cohesion_95'] = (np.nanquantile(co_all_bak,0.95, axis=0))
df_wm['cohesion_5'] = (np.nanquantile(co_all_bak,0.05, axis=0))
df_wm = df_wm[df_wm['Slp_m']>10]

df_wm.plot(column='cohesion_5',cmap='viridis', legend=True, alpha=0.7, vmin=0.0, vmax=20, ax=axs[0][0],legend_kwds={"label": "cohesion/thickness (KPa/m)", "orientation": "horizontal"},)
cx.add_basemap(axs[0][0],source='NASAGIBS.ASTER_GDEM_Greyscale_Shaded_Relief')
axs[0][0].set_axis_off()
axs[0][0].add_artist(ScaleBar(1))

df_wm.plot(column='cohesion',cmap='viridis',legend=True, alpha=0.7, vmin=0.0, vmax=20, ax=axs[0][1],legend_kwds={"label": "cohesion/thickness (KPa/m)", "orientation": "horizontal"},)
cx.add_basemap(axs[0][1],source='NASAGIBS.ASTER_GDEM_Greyscale_Shaded_Relief')
axs[0][1].set_axis_off()

df_wm.plot(column='cohesion_95',cmap='viridis',legend=True, alpha=0.7, vmin=0.0, vmax=20, ax=axs[1][0],legend_kwds={"label": "cohesion/thickness (KPa/m)", "orientation": "horizontal"},)
cx.add_basemap(axs[1][0],source='NASAGIBS.ASTER_GDEM_Greyscale_Shaded_Relief')
axs[1][0].set_axis_off()

axs[1][1].boxplot(co_all_bak[~np.isnan(co_all_bak)],0, '')
# axs[1][1].set_aspect('equal')
plt.tight_layout()
plt.savefig("PINNPlotsGallen/Cohesionv5.pdf")
plt.show()

In [ ]:
import contextily as cx
df_wm = df.to_crs(epsg=3857)
if_all_d = np.rad2deg(if_all)
if_all_d[if_all_d==0.0] = np.nan
df_wm['internalfriction'] = np.nanmedian(if_all_d,axis=0)
df_wm['internalfriction_95'] = np.nanquantile(if_all_d,0.95,axis=0)
df_wm['internalfriction_5'] = np.nanquantile(if_all_d,0.05,axis=0)

df_wm = df_wm[df_wm['Slp_m']>10]

fig,axs = plt.subplots(2,2,dpi=500, figsize=(15,15))
df_wm.plot(column='internalfriction_5',cmap='magma', legend=True, alpha=0.7, vmin=20, vmax=50, ax=axs[0][0],legend_kwds={"label": "Internal Friction Angle ($^\circ$)", "orientation": "horizontal"},)
cx.add_basemap(axs[0][0],source='NASAGIBS.ASTER_GDEM_Greyscale_Shaded_Relief')
axs[0][0].set_axis_off()
axs[0][0].add_artist(ScaleBar(1))

df_wm.plot(column='internalfriction',cmap='magma',legend=True, alpha=0.7, vmin=20, vmax=50, ax=axs[0][1],legend_kwds={"label": "Internal Friction Angle ($^\circ$)", "orientation": "horizontal"},)
cx.add_basemap(axs[0][1],source='NASAGIBS.ASTER_GDEM_Greyscale_Shaded_Relief')
axs[0][1].set_axis_off()

df_wm.plot(column='internalfriction_95',cmap='magma',legend=True, alpha=0.7, vmin=20, vmax=50, ax=axs[1][0],legend_kwds={"label": "Internal Friction Angle ($^\circ$)", "orientation": "horizontal"},)
cx.add_basemap(axs[1][0],source='NASAGIBS.ASTER_GDEM_Greyscale_Shaded_Relief')
axs[1][0].set_axis_off()

axs[1][1].boxplot(if_all_d[~np.isnan(if_all_d)],0, '')

plt.savefig("PINNPlotsGallen/Frictionv5.pdf")
plt.show()